In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns #for beatiful visualizations
%matplotlib inline 
import scipy.stats as scs #for statistics
import operator
from scipy.stats import chi2_contingency
import matplotlib.ticker as ticker

Dans cette partie nous allons efféctuer les transformation des nos variables de sortie et effetuer diverses transformations possible pour notre travil

In [2]:
dataset=pd.read_csv("DatasetOut.csv",index_col="Unnamed: 0")

In [3]:
dataset.shape

(9606, 9)

In [13]:
dataset.columns

Index([u'IDENTIFICATION', u'ACADYEAR', u'PERC1', u'MENT1', u'PERC2', u'MENT2',
       u'FAC', u'OPT', u'PROM'],
      dtype='object')

pour une premierre iteration nous allons travailler avec les données de la facultée de médecine et esuite ettendre notre modèlz aux autres facultées

In [4]:
medeDataset=dataset.loc[dataset.FAC=='FM']

In [5]:
medeDataset.head(5)

,IDENTIFICATION,ACADYEAR,PERC1,MENT1,PERC2,MENT2,FAC,OPT,PROM
34,1670,2014-2015,65.099998,S,NaN,NaN,FM,Bio-Méd,G1
35,1670,2015-2016,57.099998,A,59.200001,S,FM,Bio-Méd,G2
6871,9841,2014-2015,NaN,AA,NaN,AA,FM,Bio-Méd,G1
6917,9867,2014-2015,NaN,AA,59.200001,S,FM,Bio-Méd,G1
6918,9867,2015-2016,NaN,AA,NaN,A,FM,Bio-Méd,G2


In [6]:
medeDataset.rename(columns={'IDENTIFICATION':'ID'},inplace=True)

/Users/espyMur/Desktop/Memory-WorkingDir/memoryVenv/lib/python2.7/site-packages/pandas/core/frame.py:2834: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


Maitenant nous allons travailler sur le groupement des nos données et la creation des nouvelles variables , après cetrainnes rechreches nous avons pu conclure que pour mesurer les performances des étudiants à l'université une des métriques utilisé est le GPA(culmutative grade point average) [site note from atricles] ainsi nous allons calculer la moyenne des points des étudiants , ensuite nous calculerons les éches , ensuite pourcentage de distinctions enfin nous evaluerons le pourcentage des satisfactions , à coté de cela nous allons ajouter le nombre des fois qu'il est passé en prémiere session.

In [7]:
dataset.MENT2.value_counts().index

Index([u'S', u'A', u'AA', u'D', u'NAF', u'ADM', u'ADSTM', u'ADST', u'ADTFC',
       u'ANAF', u'AR', u'ADSTFC', u'GD'],
      dtype='object')

In [8]:
SuccefullMent=['S']
PartialSuccess=['ADM', 'ADTFC', 'ADSTM', 'ADSTFC', 'ADST']
Distinction=['D','GD']
FailSession=['AA','AR','nan']
FailureMention=['A','NAF','ANAF']

In [9]:
def calculateSucess(x):
    """this function will check if a student pass of fail in an aacademic year"""
    if str(x['MENT2']) in FailSession+FailureMention and (str(x['MENT1']) not in SuccefullMent+PartialSuccess+Distinction):
        return pd.Series(True)
    else :
        return pd.Series(False)

In [10]:
Echec=medeDataset.apply(calculateSucess,axis=1)

In [53]:
Echec[:6]

,0
34,False
35,False
6871,True
6917,False
6918,True
6937,False


In [54]:
medeDataset[:6]

,ID,ACADYEAR,PERC1,MENT1,PERC2,MENT2,FAC,OPT,PROM
34,1670,2014-2015,65.099998,S,NaN,NaN,FM,Bio-Méd,G1
35,1670,2015-2016,57.099998,A,59.200001,S,FM,Bio-Méd,G2
6871,9841,2014-2015,NaN,AA,NaN,AA,FM,Bio-Méd,G1
6917,9867,2014-2015,NaN,AA,59.200001,S,FM,Bio-Méd,G1
6918,9867,2015-2016,NaN,AA,NaN,A,FM,Bio-Méd,G2
6937,9879,2014-2015,58.200001,A,64.199997,S,FM,Bio-Méd,G1


In [11]:
def calculateSatistisfation(x):
    """this function will check if a student pass of fail in an aacademic year"""
    if x['MENT1'] in SuccefullMent+PartialSuccess or x['MENT2']in SuccefullMent+PartialSuccess:
        return pd.Series(True)
    else :
        return pd.Series(False)

In [12]:
Satisfaction=medeDataset.apply(calculateSatistisfation,axis=1)

In [13]:
def calculateDistinCtion(x):
    """this function will check if a student pass of fail in an aacademic year"""
    if x['MENT1'] in Distinction or x['MENT2']in Distinction:
        return pd.Series(True)
    else :
        return pd.Series(False)

In [14]:
DistinctionStuden=medeDataset.apply(calculateDistinCtion,axis=1)

In [15]:
DistinctionStuden.loc[DistinctionStuden.get(0)==True]

,0
7358,True
7359,True
7483,True
7598,True
8104,True
8711,True
8713,True
9185,True


In [16]:
medeDataset.loc[DistinctionStuden.get(0)==True]

,ID,ACADYEAR,PERC1,MENT1,PERC2,MENT2,FAC,OPT,PROM
7358,10126,2014-2015,71.500000,D,NaN,NaN,FM,Bio-Méd,G1
7359,10126,2015-2016,72.500000,D,NaN,NaN,FM,Bio-Méd,G2
7483,10197,2015-2016,73.599998,D,NaN,NaN,FM,Bio-Méd,G2
7598,10263,2015-2016,72.400002,D,NaN,NaN,FM,Bio-Méd,G2
8104,10577,2015-2016,73.300003,D,NaN,NaN,FM,Bio-Méd,G2
8711,11269,2015-2016,74.000000,D,NaN,NaN,FM,Bio-Méd,G1
8713,11272,2015-2016,71.699997,D,NaN,NaN,FM,Bio-Méd,G1
9185,11767,2015-2016,NaN,AA,70.300003,D,FM,Bio-Méd,G1


In [17]:
def calculatePass1estSes(x):
    """this function will check if a student pass of fail in an aacademic year"""
    if x['MENT1'] in SuccefullMent+PartialSuccess+Distinction:
        return pd.Series(True)
    else :
        return pd.Series(False)

In [18]:
PassAstSess=medeDataset.apply(calculatePass1estSes,axis=1)

In [19]:
def calculatePourcenatge(x):
    """this function will check if a student pass of fail in an aacademic year"""
    if x['MENT1'] in SuccefullMent+Distinction:
        return pd.Series(x['PERC1'])
    elif x['MENT2'] in SuccefullMent+Distinction :
        return pd.Series(x['PERC2'])
    else:
        return pd.Series(x['PERC2'])

In [20]:
FinalGrade=medeDataset.apply(calculatePourcenatge,axis=1)

In [68]:
PassAstSess

,0
34,True
35,False
6871,False
6917,False
6918,False
6937,False
6938,True
6949,True
6950,False
6955,False


In [21]:
medeDataset.loc[FinalGrade.isnull().get(0)==True]

,ID,ACADYEAR,PERC1,MENT1,PERC2,MENT2,FAC,OPT,PROM
6871,9841,2014-2015,NaN,AA,NaN,AA,FM,Bio-Méd,G1
6918,9867,2015-2016,NaN,AA,NaN,A,FM,Bio-Méd,G2
7087,9965,2014-2015,NaN,AA,NaN,AA,FM,Bio-Méd,G1
7102,9975,2014-2015,NaN,AA,NaN,AA,FM,Bio-Méd,G1
7116,9983,2014-2015,NaN,AA,NaN,AA,FM,Bio-Méd,G1
7166,10012,2014-2015,NaN,AA,NaN,AA,FM,Bio-Méd,G1
7337,10114,2014-2015,NaN,AA,NaN,AA,FM,Bio-Méd,G1
7398,10150,2014-2015,NaN,AA,NaN,AA,FM,Bio-Méd,G1
7408,10155,2015-2016,NaN,AA,NaN,A,FM,Bio-Méd,G2
7418,10161,2014-2015,NaN,AA,NaN,AA,FM,Bio-Méd,G1


In [22]:
medeDataset['Satisfaction']=Satisfaction
medeDataset['echec']=Echec
medeDataset['distinction']=DistinctionStuden
medeDataset['Pass1stSession']=PassAstSess
medeDataset['PercFinal']=FinalGrade

/Users/espyMur/Desktop/Memory-WorkingDir/memoryVenv/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/espyMur/Desktop/Memory-WorkingDir/memoryVenv/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/espyMur/Desktop/Memory-WorkingDir/memoryVenv/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [23]:
medeDataset.loc[:65]

,ID,ACADYEAR,PERC1,MENT1,PERC2,MENT2,FAC,OPT,PROM,Satisfaction,echec,distinction,Pass1stSession,PercFinal
34,1670,2014-2015,65.099998,S,NaN,NaN,FM,Bio-Méd,G1,True,False,False,True,65.099998
35,1670,2015-2016,57.099998,A,59.200001,S,FM,Bio-Méd,G2,True,False,False,False,59.200001


In [24]:
medeDataset.PercFinal=medeDataset.PercFinal.apply(lambda x: np.random.choice([x for x in range(45,53)]) if (np.isnan(x)) else x)

/Users/espyMur/Desktop/Memory-WorkingDir/memoryVenv/lib/python2.7/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [25]:
len(medeDataset.loc[medeDataset.Pass1stSession==True])

97

In [26]:
medeDataset.head(8)

,ID,ACADYEAR,PERC1,MENT1,PERC2,MENT2,FAC,OPT,PROM,Satisfaction,echec,distinction,Pass1stSession,PercFinal
34,1670,2014-2015,65.099998,S,NaN,NaN,FM,Bio-Méd,G1,True,False,False,True,65.099998
35,1670,2015-2016,57.099998,A,59.200001,S,FM,Bio-Méd,G2,True,False,False,False,59.200001
6871,9841,2014-2015,NaN,AA,NaN,AA,FM,Bio-Méd,G1,False,True,False,False,52.000000
6917,9867,2014-2015,NaN,AA,59.200001,S,FM,Bio-Méd,G1,True,False,False,False,59.200001
6918,9867,2015-2016,NaN,AA,NaN,A,FM,Bio-Méd,G2,False,True,False,False,49.000000
6937,9879,2014-2015,58.200001,A,64.199997,S,FM,Bio-Méd,G1,True,False,False,False,64.199997
6938,9879,2015-2016,58.900002,S,NaN,NaN,FM,Bio-Méd,G2,True,False,False,True,58.900002
6949,9887,2014-2015,60.000000,S,NaN,NaN,FM,Bio-Méd,G1,True,False,False,True,60.000000


continous avec le calcul du ratio.

In [44]:
medeDataset.loc[medeDataset.echec==True]['']

,ID,ACADYEAR,PERC1,MENT1,PERC2,MENT2,FAC,OPT,PROM,Satisfaction,echec,distinction,Pass1stSession,PercFinal
6871,9841,2014-2015,NaN,AA,NaN,AA,FM,Bio-Méd,G1,False,True,False,False,52.000000
6918,9867,2015-2016,NaN,AA,NaN,A,FM,Bio-Méd,G2,False,True,False,False,49.000000
6987,9909,2014-2015,NaN,AA,52.000000,A,FM,Bio-Méd,G1,False,True,False,False,52.000000
7087,9965,2014-2015,NaN,AA,NaN,AA,FM,Bio-Méd,G1,False,True,False,False,47.000000
7102,9975,2014-2015,NaN,AA,NaN,AA,FM,Bio-Méd,G1,False,True,False,False,51.000000
7116,9983,2014-2015,NaN,AA,NaN,AA,FM,Bio-Méd,G1,False,True,False,False,47.000000
7132,9993,2014-2015,NaN,AA,52.700001,A,FM,Bio-Méd,G1,False,True,False,False,52.700001
7166,10012,2014-2015,NaN,AA,NaN,AA,FM,Bio-Méd,G1,False,True,False,False,46.000000
7337,10114,2014-2015,NaN,AA,NaN,AA,FM,Bio-Méd,G1,False,True,False,False,48.000000
7362,10128,2014-2015,NaN,AA,49.799999,A,FM,Bio-Méd,G1,False,True,False,False,49.799999


In [45]:
medeDataset.columns

Index([u'ID', u'ACADYEAR', u'PERC1', u'MENT1', u'PERC2', u'MENT2', u'FAC',
       u'OPT', u'PROM', u'Satisfaction', u'echec', u'distinction',
       u'Pass1stSession', u'PercFinal'],
      dtype='object')

In [69]:
def calculateReussiteDistin(x):
    """this function will check if a student pass of fail in an aacademic year"""
    if x['echec']==False and x['distinction']==True and x['Satisfaction']==False:
        """ceux qui ont distinguer"""
        if x['Pass1stSession']==True:
            """en premiere session"""
            return pd.Series(True)
        elif x['Pass1stSession']==False:
            """en seconde session"""
            return pd.Series(True)
    
    elif x['echec']==False and x['distinction']==False and x['Satisfaction']==True:
        if x['Pass1stSession']==True:
            """en premiere session"""
            return pd.Series(False)
        elif x['Pass1stSession']==False:
            """en seconde session"""
            return pd.Series(False)
    else :
        return pd.Series(False)

In [64]:
def calculateReussite1Sessi(x):
    """this function will check if a student pass of fail in an aacademic year"""
    if x['echec']==False and x['distinction']==True and x['Satisfaction']==False:
        """ceux qui ont distinguer"""
        if x['Pass1stSession']==True:
            """en premiere session"""
            return pd.Series(True)
        elif x['Pass1stSession']==False:
            """en seconde session"""
            return pd.Series(False)
    
    elif x['echec']==False and x['distinction']==False and x['Satisfaction']==True:
        if x['Pass1stSession']==True:
            """en premiere session"""
            return pd.Series(True)
        elif x['Pass1stSession']==False:
            """en seconde session"""
            return pd.Series(False)
    else :
        return pd.Series(False)   

In [70]:
ReusiteDistinc=medeDataset.apply(calculateReussiteDistin,axis=1)


In [72]:
medeDataset.loc[ReusiteDistinc.get(0)==True]

,ID,ACADYEAR,PERC1,MENT1,PERC2,MENT2,FAC,OPT,PROM,Satisfaction,echec,distinction,Pass1stSession,PercFinal
7358,10126,2014-2015,71.500000,D,NaN,NaN,FM,Bio-Méd,G1,False,False,True,True,71.500000
7359,10126,2015-2016,72.500000,D,NaN,NaN,FM,Bio-Méd,G2,False,False,True,True,72.500000
7483,10197,2015-2016,73.599998,D,NaN,NaN,FM,Bio-Méd,G2,False,False,True,True,73.599998
7598,10263,2015-2016,72.400002,D,NaN,NaN,FM,Bio-Méd,G2,False,False,True,True,72.400002
8104,10577,2015-2016,73.300003,D,NaN,NaN,FM,Bio-Méd,G2,False,False,True,True,73.300003
8711,11269,2015-2016,74.000000,D,NaN,NaN,FM,Bio-Méd,G1,False,False,True,True,74.000000
8713,11272,2015-2016,71.699997,D,NaN,NaN,FM,Bio-Méd,G1,False,False,True,True,71.699997
9185,11767,2015-2016,NaN,AA,70.300003,D,FM,Bio-Méd,G1,False,False,True,False,70.300003


In [74]:
Reusite1erSess=medeDataset.apply(calculateReussite1Sessi,axis=1)

In [85]:
type(Reusite1erSess)

pandas.core.frame.DataFrame

In [89]:
medeDataset['Reusite1erSess']=Reusite1erSess

/Users/espyMur/Desktop/Memory-WorkingDir/memoryVenv/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [91]:
medeDataset['ReusiteDistinc']=ReusiteDistinc

/Users/espyMur/Desktop/Memory-WorkingDir/memoryVenv/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [93]:
medeDataset.columns

Index([u'ID', u'ACADYEAR', u'PERC1', u'MENT1', u'PERC2', u'MENT2', u'FAC',
       u'OPT', u'PROM', u'Satisfaction', u'echec', u'distinction',
       u'Pass1stSession', u'PercFinal', u'Reusite1erSess', u'ReusiteDistinc'],
      dtype='object')

In [81]:
Reusite1erSess.loc[medeDataset.echec==True].get(0).value_counts()

False    78
Name: 0, dtype: int64

les nouvelles collones cree contiennent les rations:
    echec: Nous dit si l'edudiant a échouer dans une année académique
    resuite: Avec Distinction ous Satisfaction
    Session: 1ere Session ou pAs 
    Pourcentage Final:NB(Les étudiant qui sont AA au final : bref ceux qui n'ont pas de pourcentage on été remplacé avec un nobre aleatoire compris entre 45et 52')

In [125]:
medeDataset.columns

Index([u'ID', u'ACADYEAR', u'PERC1', u'MENT1', u'PERC2', u'MENT2', u'FAC',
       u'OPT', u'PROM', u'Satisfaction', u'echec', u'distinction',
       u'Pass1stSession', u'PercFinal'],
      dtype='object')

In [98]:
 def f(x):
    A=list(x['echec'])
    D=list(x['ReusiteDistinc'])
    S1=list(x['Reusite1erSess'])
    return pd.Series(dict(
     echec=int(A.count(True)),
     distinction=int(D.count(True)),
     Pass1stSession=int(S1.count(True)),
     CGPA=np.mean(x['PercFinal']),
     Nbre=int(len(x['ACADYEAR']))
    ))

In [99]:
medeDatasetRatio=medeDataset.groupby('ID').apply(f).reset_index()

In [100]:
medeDatasetRatio.head(5)

,ID,CGPA,Nbre,Pass1stSession,distinction,echec
0,1670,62.150000,2.0,1.0,0.0,0.0
1,9841,52.000000,1.0,0.0,0.0,1.0
2,9867,54.100000,2.0,0.0,0.0,1.0
3,9879,61.549999,2.0,1.0,0.0,0.0
4,9887,59.450001,2.0,1.0,0.0,0.0


In [101]:
medeDatasetRatio.loc[:,'Pass1stSession']=medeDatasetRatio.Pass1stSession/medeDatasetRatio.Nbre

In [102]:
medeDatasetRatio.loc[:,'distinction']=medeDatasetRatio.distinction/medeDatasetRatio.Nbre
medeDatasetRatio.loc[:,'echec']=medeDatasetRatio.echec/medeDatasetRatio.Nbre

In [103]:
medeDatasetRatio.head(5)

,ID,CGPA,Nbre,Pass1stSession,distinction,echec
0,1670,62.150000,2.0,0.5,0.0,0.0
1,9841,52.000000,1.0,0.0,0.0,1.0
2,9867,54.100000,2.0,0.0,0.0,0.5
3,9879,61.549999,2.0,0.5,0.0,0.0
4,9887,59.450001,2.0,0.5,0.0,0.0


Voila voici enfin notre ensemble de soertie : voici une description des colonnes qu'il comprend :
    - CGPA : La moyenne des pourcentages final Obtenu par l'etudiant
    - Pass1stSession: Nombré d'année que l'étudiant est passé en en 1 ere session divisé par les nombré d'années passées à la fac
    - distinction: Nombré d'années que l'étudiant a distingué divisé par les nombré d'années passées à la fac
    - echec: Nombré d'années que l'étudiant a échoué divisé par les nombré d'années passées à la fac


In [113]:
medeDatasetRatio.rename(columns={'Pass1stSession':'Ratio1ss','distinction':'RatioDist','echec':'RatioEchec'},inplace=True)

In [117]:
medeDataset=pd.merge(medeDataset,medeDatasetRatio[['ID','Ratio1ss','RatioDist','RatioEchec']],on='ID')

In [124]:
dasetOutMed=medeDatasetRatio

In [122]:
medeDataset[[u'ID', u'echec',
        u'PercFinal', u'Reusite1erSess', u'ReusiteDistinc',
       u'Ratio1ss', u'RatioDist', u'RatioEchec']]

,ID,echec,PercFinal,Reusite1erSess,ReusiteDistinc,Ratio1ss,RatioDist,RatioEchec
0,1670,False,65.099998,True,False,0.5,0.0,0.0
1,1670,False,59.200001,False,False,0.5,0.0,0.0
2,9841,True,52.000000,False,False,0.0,0.0,1.0
3,9867,False,59.200001,False,False,0.0,0.0,0.5
4,9867,True,49.000000,False,False,0.0,0.0,0.5
5,9879,False,64.199997,False,False,0.5,0.0,0.0
6,9879,False,58.900002,True,False,0.5,0.0,0.0
7,9887,False,60.000000,True,False,0.5,0.0,0.0
8,9887,False,58.900002,False,False,0.5,0.0,0.0
9,9891,False,56.299999,False,False,0.0,0.0,0.0


In [126]:
dasetOutMed.to_csv('DatasetOutMed.csv')